In [2]:
# (a)
import boto3
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import recall_score

# Defining s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'gabrielferreira-data-455-bucket'
bucket = s3.Bucket(bucket_name)

# Defining the file to be read from s3 bucket
key_file = 'framingham.csv'

bucket_object = bucket.Object(key_file)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# Reading csv
heart = pd.read_csv(file_content_stream)

# (b)
heart = heart.dropna()
heart.head(1)

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0


In [8]:
# Defining predictors and target variable
X = heart[['male', 'age', 'currentSmoker', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

# Splitting the dataset into predictors and target variables
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [15]:
# Transforming predictors variables to 0-1 scale
scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns = X.columns)

In [16]:
## LASSO CV
lasso_cv = LassoCV(cv = 5).fit(X_train, Y_train)
lasso_cv.alpha_

# Building LASSO regression
lasso_md = Lasso(alpha = lasso_cv.alpha_).fit(X_train, Y_train)
lasso_md.coef_

array([0.06818806, 0.28407088, 0.04586193, 0.        , 0.48175676,
       0.        , 0.        , 0.27538595])

In [17]:
X_train_lasso = X_train[['male', 'age', 'currentSmoker', 'sysBP', 'glucose']]
X_test_lasso = X_test[['male', 'age', 'currentSmoker', 'sysBP', 'glucose']]

# Logistic Regression
logit_md = LogisticRegression().fit(X_train_lasso, Y_train)

# Predicting on test
pred1 = logit_md.predict_proba(X_test_lasso)[:,1]
pred1 = np.where(pred1 < 0.1, 0, 1)

# Computing the recall
recall_score(Y_test, pred1)

0.8928571428571429

In [18]:
# Random forest classifier
RF = RandomForestClassifier(n_estimators = 500).fit(X_train, Y_train)

# Extracting importance
RF.feature_importances_

array([0.02456363, 0.15141812, 0.01959902, 0.16628495, 0.18914261,
       0.17289656, 0.12321649, 0.15287863])

In [22]:
X_train_RF = X_train[['age', 'totChol', 'sysBP', 'BMI', 'glucose']]
X_test_RF = X_test[['age', 'totChol', 'sysBP', 'BMI', 'glucose']]

# Building Random Forest Model
RF = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train_RF, Y_train)

# Predicting on test dataset
pred2 = RF.predict_proba(X_test_RF)[:,1]
pred2 = np.where(pred2 < 0.1, 0, 1)

recall_score(Y_test, pred2)

0.9107142857142857